In [49]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer


In [50]:
df=pd.read_csv('ModApte_test.csv')

In [51]:
def series_to_list(df,column_name):
    result=df[column_name].replace({' ':''},regex=True)
    result.replace({'\\n':''},regex=True,inplace=True)
    result.replace({'\'\'':'\',\''},regex=True,inplace=True)
    return result.apply(ast.literal_eval)

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3299 entries, 0 to 3298
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         3023 non-null   object
 1   text_type    3299 non-null   object
 2   topics       3299 non-null   object
 3   lewis_split  3299 non-null   object
 4   cgis_split   3299 non-null   object
 5   old_id       3299 non-null   object
 6   new_id       3299 non-null   object
 7   places       3299 non-null   object
 8   people       3299 non-null   object
 9   orgs         3299 non-null   object
 10  exchanges    3299 non-null   object
 11  date         3299 non-null   object
 12  title        3285 non-null   object
dtypes: object(13)
memory usage: 335.2+ KB


In [53]:
df.dtypes

text           object
text_type      object
topics         object
lewis_split    object
cgis_split     object
old_id         object
new_id         object
places         object
people         object
orgs           object
exchanges      object
date           object
title          object
dtype: object

In [54]:
df.head()

,text,text_type,topics,lewis_split,cgis_split,old_id,new_id,places,people,orgs,exchanges,date,title
0,Mounting trade friction between the\nU.S. And ...,"""NORM""",['trade'],"""TEST""","""TRAINING-SET""","""3809""","""14826""",['hong-kong' 'usa' 'japan' 'taiwan' 'malaysia'...,[],[],[],8-APR-1987 01:03:47.52,ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT
1,A survey of 19 provinces and seven cities\nsho...,"""NORM""",['grain'],"""TEST""","""TRAINING-SET""","""3811""","""14828""",['china'],[],[],[],8-APR-1987 01:19:17.29,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS
2,The Ministry of International Trade and\nIndus...,"""NORM""",['crude' 'nat-gas'],"""TEST""","""TRAINING-SET""","""4356""","""14829""",['japan'],[],[],[],8-APR-1987 01:22:17.25,JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWARDS
3,Thailand's trade deficit widened to 4.5\nbilli...,"""NORM""",['trade' 'grain' 'rice' 'corn' 'sugar' 'tin' '...,"""TEST""","""TRAINING-SET""","""3815""","""14832""",['thailand'],[],[],[],8-APR-1987 01:45:09.09,THAI TRADE DEFICIT WIDENS IN FIRST QUARTER
4,Indonesia expects crude palm oil (CPO)\nprices...,"""NORM""",['veg-oil' 'palm-oil'],"""TEST""","""TRAINING-SET""","""3816""","""14833""",['indonesia' 'malaysia'],[],[],[],8-APR-1987 01:48:20.11,INDONESIA SEES CPO PRICE RISING SHARPLY


In [55]:
foods = ['coconut', 'cotton-oil', 'sorghum', 'orange', 'rice', 'soybean', 'sun-meal', 
    'oilseed', 'sugar', 'hog', 'coffee', 'groundnut', 'sunseed', 'sun-oil', 'rye', 
    'lin-oil', 'copra-cake', 'potato', 'barley', 'tea', 'meal-feed', 'coconut-oil', 
    'palmkernel', 'cottonseed', 'castor-oil', 'l-cattle', 'livestock', 'soy-oil', 
    'rape-oil', 'palm-oil', 'cocoa', 'cotton', 'wheat', 'corn', 'f-cattle', 'grain', 
    'soy-meal', 'oat', 'groundnut-oil', 'veg-oil','rapeseed']
for food in foods:
    df['topics'].replace({'\''+food+'\'':'\'food\''},regex=True,inplace=True)

C:\Users\theinrich\AppData\Local\Temp\ipykernel_19132\3574597661.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['topics'].replace({'\''+food+'\'':'\'food\''},regex=True,inplace=True)


In [56]:
resources = ['platinum', 'lead', 'nickel', 'strategic-metal', 'copper', 'palladium', 'gold', 
    'zinc', 'tin', 'iron-steel', 'alum', 'silver', 'nat-gas', 'rubber', 'pet-chem', 'fuel', 'crude','lumber','propane']
for resource in resources:
    df['topics'].replace({'\''+resource+'\'':'\'resource\''},regex=True,inplace=True)

In [57]:
finances = ['money-supply', 'dlr', 'nkr', 'lei', 'yen', 'dfl', 'sfr', 'cpi', 'instal-debt', 
    'money-fx', 'gnp', 'interest', 'income', 'dmk', 'rand', 'bop', 'reserves', 'nzdlr','acq']
for finance in finances:
    df['topics'].replace({'\''+finance+'\'':'\'finance\''},regex=True,inplace=True)

In [58]:
personal_finances = ['housing','jobs','earn']
for personal_finance in personal_finances:
    df['topics'].replace({'\''+personal_finance+'\'':'\'personal_finance\''},regex=True,inplace=True)

In [59]:
transports = ['jet', 'ship']
for transport in transports:
    df['topics'].replace({'\''+transport+'\'':'\'transport\''},regex=True,inplace=True)

In [60]:
removes = ['gas', 'heat', 'trade', 'retail', 'carcass', 'cpu', 'wpi', 'naphtha', 'ipi']
for remove in removes:
    df['topics'].replace({'\''+remove+'\'':''},regex=True,inplace=True)

In [61]:
df['topics'].replace({' ':''},regex=True,inplace=True)

In [62]:
df['topics']=series_to_list(df,'topics')
headers=['places','people','orgs','exchanges']
for header in headers:
    df[header]=series_to_list(df,header)
    print(df[header].value_counts())


places
[usa]                                                    1786
[]                                                        228
[uk]                                                      142
[canada]                                                  132
[japan]                                                    64
                                                         ... 
[japan, usa, uk, west-germany, france, italy, canada]       1
[new-zealand, fiji]                                         1
[iran, ussr, cyprus]                                        1
[japan, west-germany, usa]                                  1
[hong-kong, japan, india, pakistan, iran, iraq]             1
Name: count, Length: 359, dtype: int64
people
[]                      3127
[reagan]                  13
[james-baker]             11
[lyng]                    10
[miyazawa]                10
                        ... 
[delors]                   1
[reagan, verity]           1
[yeutter, de-clercq]       1
[per

In [63]:

df = df.join(pd.crosstab((s:=df['places'].explode()).index, s).add_prefix('places'+'_'))
df.drop(columns=headers,inplace=True)
df.drop(columns=['lewis_split','cgis_split','old_id','new_id'],inplace=True)

In [102]:
df.head()

,text,text_type,topics,date,title,places_algeria,places_argentina,places_australia,places_austria,places_bahrain,...,places_usa,places_ussr,places_venezuela,places_west-germany,places_yemen-arab-republic,places_yemen-demo-republic,places_yugoslavia,places_zaire,places_zambia,places_zimbabwe
1,A survey of 19 provinces and seven cities\nsho...,NORM,food,8-APR-1987 01:19:17.29,CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,"Tug crews in New South Wales (NSW),\nVictoria ...",NORM,transport,8-APR-1987 02:42:14.86,AUSTRALIAN FOREIGN SHIP BAN ENDS BUT NSW PORTS...,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Western Mining Corp Holdings Ltd\n&lt;WMNG.S> ...,NORM,resource,8-APR-1987 03:21:39.74,WESTERN MINING TO OPEN NEW GOLD MINE IN AUSTRALIA,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,Sumitomo Bank Ltd &lt;SUMI.T> is certain to\nl...,NORM,finance,8-APR-1987 03:29:34.51,SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,Mines and Energy Minister Subroto\nconfirmed I...,NORM,resource,8-APR-1987 03:45:20.53,SUBROTO SAYS INDONESIA SUPPORTS TIN PACT EXTEN...,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
for col  in [col for col in df if col.startswith('places_')]:
    df[col].fillna(value=0,inplace=True)
df['text'].dropna(inplace=True)
df=df[df['topics'].str.len()==1]
df['topics']=df['topics'].apply(lambda x : x[0])

C:\Users\theinrich\AppData\Local\Temp\ipykernel_19132\1352241873.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(value=0,inplace=True)


In [92]:
df['text_type'].replace({'"':''},regex=True,inplace=True)


C:\Users\theinrich\AppData\Local\Temp\ipykernel_19132\187516221.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['text_type'].replace({'"':''},regex=True,inplace=True)


In [103]:
df['date'].dt.day

AttributeError: Can only use .dt accessor with datetimelike values

In [88]:
df['text_type'].value_counts()

text_type
"NORM"      2272
"BRIEF"      222
"UNPROC"      12
Name: count, dtype: int64

In [83]:
test=test[test.str.len()==1]
test.count()

np.int64(2808)

In [132]:
unique_entries = set()
for sublist in test[test.str.len()==1]:
    unique_entries.update(sublist)

print(unique_entries)

{'resource', 'personal_finance', 'food', 'transport', 'finance'}


In [84]:
test.value_counts()


topics
[personal_finance]    1098
[finance]             1083
[resource]             296
[food]                 294
[transport]             37
Name: count, dtype: int64